In [1]:
import numpy as np
import pandas as pd
import nltk
import json
import re
import csv
from tqdm import tqdm
pd.set_option('display.max_colwidth', 300)

data = []

with open("input/booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
        data.append(row)

from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

16559it [00:00, 24315.79it/s]


In [2]:
book_index = []
book_id = []
book_author = []
book_name = []
summary = []
genre = []
a = 1
for i in tqdm(data):
    book_index.append(a)
    a += 1
    book_id.append(i[0])
    book_name.append(i[2])
    book_author.append(i[3])
    genre.append(i[5])
    summary.append(i[6])

df = pd.DataFrame({'index': book_index, 'id': book_id, 'title': book_name, 'author': book_author, 'genre': genre, 'summary': summary})

100%|██████████| 16559/16559 [00:00<00:00, 670924.95it/s]


In [3]:
df.head()

,index,id,title,author,genre,summary
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
3,4,1756,An Enquiry Concerning Human Understanding,David Hume,,"The argument of the Enquiry proceeds by a series of incremental steps, separated into chapters which logically succeed one another. After expounding his epistemology, Hume explains how to apply his principles to specific topics. In the first section of the Enquiry, Hume provides a rough introdu..."
4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."


In [4]:
df = df.drop(df[df['genre'] == ''].index)
df = df.drop(df[df['summary'] == ''].index)

genres_cleaned = []
for i in df['genre']:
    genres_cleaned.append(list(json.loads(i).values()))
df['genre'] = genres_cleaned
df['genrestring'] = df['genre'].apply(lambda x: ' '.join(x))

In [6]:
text_df = df[['title', 'author', 'genrestring', 'summary']]
text_df.head()

,title,author,genrestring,summary
0,Animal Farm,George Orwell,Roman à clef Satire Children's literature Speculative fiction Fiction,"Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,A Clockwork Orange,Anthony Burgess,Science Fiction Novella Speculative fiction Utopian and dystopian fiction Satire Fiction,"Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,The Plague,Albert Camus,Existentialism Fiction Absurdist fiction Novel,"The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
4,A Fire Upon the Deep,Vernor Vinge,Hard science fiction Science Fiction Speculative fiction Fantasy Fiction,"The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."
5,All Quiet on the Western Front,Erich Maria Remarque,War novel Roman à clef,"The book tells the story of Paul Bäumer, a German soldier who—urged on by his school teacher—joins the German army shortly after the start of World War I. Bäumer arrives at the Western Front with his friends and schoolmates (Tjaden, Müller, Kropp and a number of other characters). There they me..."


In [9]:
# vectorize text
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(text_df, show_progress_bar=True)

Downloading: 100%|██████████| 190/190 [00:00<00:00, 71.5kB/s]
Downloading: 100%|██████████| 3.69k/3.69k [00:00<00:00, 1.44MB/s]
Downloading: 100%|██████████| 229/229 [00:00<00:00, 74.4kB/s]
Downloading: 100%|██████████| 90.9M/90.9M [00:07<00:00, 12.2MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 22.4kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 42.3kB/s]
Downloading: 100%|██████████| 466k/466k [00:01<00:00, 377kB/s]  
Downloading: 100%|██████████| 314/314 [00:00<00:00, 69.4kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 301kB/s]  


KeyError: 2